In [1]:
import numpy as np
import pandas as pd
import re

from matplotlib import pylab as plt
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split

from  sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

In [2]:
train_path = 'dataset/hell-week-mic/train.csv'

df_train = pd.read_csv(train_path, nrows=1000)
df_train.head(10)

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state
0,T77962103,7059868985,673.0,2.131868e+14,Copy,guerrerodavid@example.org,http://garcia.org/,Gabon,📑,Cathy Cherry,...,🍣🏿🟠U👨@9.028568970658524@️bY‍t,N6♀rs%-0.17374199145886737%g2Fu7,dwLNC+-16.438253859793118+D🟠Mj👩,♀Y‍tt^-5.683708646092431^7b🎓🍣T,Q👩👨H🏾?4.495020766798911?p🎨dBV,JC🍣8Q!1.3976340938324077!Wo‍6t,QtgcM|-2.1616012233562616|srwW❇,♀👩🦐pq%7.619063527183258%👨Oet♀,‍🏿🟠👩X+-7.00274740118153+‍b🎨VK,Despair
1,O92591533,+1-288-810-2425x013,983.0,4.820055e+15,Interior and spatial designer,michaela47@example.org,http://ortiz.com/,Bermuda,🥷🏿,Austin Graham,...,🍣🏿🟠U👨@-40.77392373961654@️bY‍t,N6♀rs%3.836527507653148%g2Fu7,dwLNC+-20.503862282922828+D🟠Mj👩,♀Y‍tt^4.345119531921794^7b🎓🍣T,Q👩👨H🏾?-0.19904265803833332?p🎨dBV,JC🍣8Q!3.414072326617339!Wo‍6t,QtgcM|8.673900995107994|srwW❇,♀👩🦐pq%-0.6359934104694338%👨Oet♀,‍🏿🟠👩X+-11.039355040136286+‍b🎨VK,Worthlessness
2,O31883571,791.502.4387x7276,855.0,4.031429e+15,"Scientist, water quality",zhoffman@example.org,http://kelly.com/,Sao Tome and Principe,👩‍💻,Frank Smith,...,🍣🏿🟠U👨@32.83118091536586@️bY‍t,N6♀rs%6.673810456792015%g2Fu7,dwLNC+31.545003675536307+D🟠Mj👩,♀Y‍tt^-1.1189348865153141^7b🎓🍣T,Q👩👨H🏾?-2.109315804181183?p🎨dBV,JC🍣8Q!6.773526691273227!Wo‍6t,QtgcM|1.7184076239046218|srwW❇,♀👩🦐pq%-35.95525145090523%👨Oet♀,‍🏿🟠👩X+8.6752786215897+‍b🎨VK,Stress
3,M18080565,2522308761,806.0,3.036423e+13,Toxicologist,hprice@example.net,http://mann.info/,Korea,NaN,Austin Young,...,🍣🏿🟠U👨@-11.829454766573882@️bY‍t,N6♀rs%-10.87012476653127%g2Fu7,dwLNC+26.131133464440587+D🟠Mj👩,♀Y‍tt^2.1159115210320385^7b🎓🍣T,Q👩👨H🏾?7.373067390546165?p🎨dBV,JC🍣8Q!-0.2388097363537749!Wo‍6t,QtgcM|-5.42787813792933|srwW❇,♀👩🦐pq%24.501780969213996%👨Oet♀,‍🏿🟠👩X+-0.8456272162320895+‍b🎨VK,Fatigue
4,Q74073854,683-521-2001x423,424.0,4.610207e+15,Software engineer,michael61@example.net,https://www.mitchell.biz/,Russian Federation,💆🏿‍♂️,Kimberly Wilson,...,🍣🏿🟠U👨@10.957430264267883@️bY‍t,N6♀rs%-1.5309050040841568%g2Fu7,dwLNC+-17.670060957130605+D🟠Mj👩,♀Y‍tt^0.132418797411758^7b🎓🍣T,Q👩👨H🏾?3.6960467337932292?p🎨dBV,JC🍣8Q!4.588960052905303!Wo‍6t,QtgcM|-3.735829327461521|srwW❇,♀👩🦐pq%18.194146730985878%👨Oet♀,‍🏿🟠👩X+10.404818391481196+‍b🎨VK,Tears
5,P43985373,219.273.4726x275,428.0,2.131351e+14,Health promotion specialist,james93@example.org,https://ochoa-gutierrez.net/,Congo,🧍🏾‍♀️,Brent Hernandez,...,🍣🏿🟠U👨@17.58268623621198@️bY‍t,N6♀rs%-2.811845523465795%g2Fu7,dwLNC+3.773968437289522+D🟠Mj👩,♀Y‍tt^1.8456705455150582^7b🎓🍣T,Q👩👨H🏾?-2.7994908849184856?p🎨dBV,JC🍣8Q!-1.2433056271592056!Wo‍6t,QtgcM|-2.26513514409478|srwW❇,♀👩🦐pq%-26.64323697122899%👨Oet♀,‍🏿🟠👩X+-1.1054359483286063+‍b🎨VK,Nausea
6,I95928831,601.741.1529x4852,610.0,3.516388e+15,"Pharmacist, hospital",michael12@example.net,https://hall.com/,Turks and Caicos Islands,🦏,Mr. Todd Richards DDS,...,🍣🏿🟠U👨@-18.588666405251253@️bY‍t,N6♀rs%8.217194776982542%g2Fu7,dwLNC+-3.3867603573540714+D🟠Mj👩,♀Y‍tt^-4.764344688857977^7b🎓🍣T,Q👩👨H🏾?-1.7839891815790612?p🎨dBV,JC🍣8Q!3.1100008893905495!Wo‍6t,QtgcM|3.8792464372230127|srwW❇,♀👩🦐pq%-7.266273863482758%👨Oet♀,‍🏿🟠👩X+-0.761454161839668+‍b🎨VK,Tears
7,V14371828,505.735.3796x37094,944.0,5.662624e+11,Educational psychologist,youngjordan@example.org,http://www.cardenas.biz/,Saint Pierre and Miquelon,💿,Julie Henderson,...,🍣🏿🟠U👨@12.737872216741858@️bY‍t,N6♀rs%-3.0215000872197977%g2Fu7,dwLNC+-5.372761138249989+D🟠Mj👩,♀Y‍tt^7.167058874093296^7b🎓🍣T,Q👩👨H🏾?1.2475639802668326?p🎨dBV,JC🍣8Q!5.2498269170861125!Wo‍6t,QtgcM|5.767867801335005|srwW❇,♀👩🦐pq%-32.04786386746257%👨Oet♀,‍🏿🟠👩X+1.2147268865928584+‍b🎨VK,Pain
8,Y13708888,660.780.8534,133.0,3.029480e+13,Toxicologist,mckinneykathy@example.org,https://www.sanchez-gilbert.com/,Iceland,🤾🏼,Mr. Matthew Goodman,...,🍣🏿🟠U👨@2.8060487179477285@️bY‍t,N6♀rs%-6.085554715936928%g2Fu7,dwLNC+-12.0824224039864+D🟠Mj👩,♀Y‍tt^5.2493706147779315^7b🎓🍣T,Q👩👨H🏾?-2.90735592087888?p🎨dBV,JC🍣8Q!-7.309327293951238!Wo‍6t,QtgcM|2.8859631583166134|srwW❇,♀👩🦐pq%33.307436

In [3]:
test_path = 'dataset/hell-week-mic/test.csv'

df_test= pd.read_csv(test_path)
df_test.head(10)

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,OaMqz,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi
0,675919160,001-869-364-3240x1461,632.0,4.787566e+12,"Scientist, research (maths)",lawrencereginald@example.org,http://walker.org/,Honduras,🧜🏻‍♀️,Jean Barajas,...,Iwp🏤5!-21.248937402363666!G🚵O‍🏿,🍣🏿🟠U👨@-5.726684042857591@️bY‍t,N6♀rs%1.5777964549882526%g2Fu7,dwLNC+-8.911004387243423+D🟠Mj👩,♀Y‍tt^5.9810077922437666^7b🎓🍣T,Q👩👨H🏾?-1.2978943388453708?p🎨dBV,JC🍣8Q!-1.6739446957794175!Wo‍6t,QtgcM|1.4464588802972562|srwW❇,♀👩🦐pq%-29.42164549286103%👨Oet♀,‍🏿🟠👩X+1.7950039158754367+‍b🎨VK
1,V09461652,+1-573-696-9623x435,548.0,4.804074e+15,Higher education careers adviser,jennifer41@example.org,http://shah.com/,Netherlands Antilles,💁🏻‍♀️,Sandra Ellis,...,Iwp🏤5!-33.934398562714854!G🚵O‍🏿,🍣🏿🟠U👨@-19.149703711482513@️bY‍t,N6♀rs%-2.868717979567546%g2Fu7,dwLNC+15.453724177383293+D🟠Mj👩,♀Y‍tt^-6.21132167380491^7b🎓🍣T,Q👩👨H🏾?-0.9339788851074693?p🎨dBV,JC🍣8Q!0.001376362594596836!Wo‍6t,QtgcM|-4.159290969741395|srwW❇,♀👩🦐pq%-1.5404064055083722%👨Oet♀,‍🏿🟠👩X+1.2969428631934425+‍b🎨VK
2,S75396644,001-755-901-1494x000,808.0,4.890170e+15,Illustrator,valdezsheryl@example.net,http://barrett.com/,Lao People's Democratic Republic,👭🏼,Elizabeth Smith,...,Iwp🏤5!12.391218072873253!G🚵O‍🏿,🍣🏿🟠U👨@-14.048287304034654@️bY‍t,N6♀rs%-5.027959013789114%g2Fu7,dwLNC+6.75452704247651+D🟠Mj👩,♀Y‍tt^7.670390381027927^7b🎓🍣T,Q👩👨H🏾?0.3470814661758477?p🎨dBV,JC🍣8Q!1.407363830373458!Wo‍6t,QtgcM|6.091721410416965|srwW❇,♀👩🦐pq%-22.07040959384532%👨Oet♀,‍🏿🟠👩X+-3.7487716558978468+‍b🎨VK
3,598599835,(625)805-7487x931,9468.0,5.428366e+15,Audiological scientist,NaN,http://melendez.com/,Sierra Leone,🖲️,Allison Rodriguez,...,Iwp🏤5!11.456072353615188!G🚵O‍🏿,🍣🏿🟠U👨@-12.459836898257926@️bY‍t,N6♀rs%-3.4487706404694176%g2Fu7,dwLNC+20.40768203656919+D🟠Mj👩,♀Y‍tt^-1.6018121132255914^7b🎓🍣T,Q👩👨H🏾?1.3550954935361417?p🎨dBV,JC🍣8Q!0.9793080156219274!Wo‍6t,QtgcM|-1.4099964218763166|srwW❇,♀👩🦐pq%2.1205798212068476%👨Oet♀,‍🏿🟠👩X+-3.9241064330693742+‍b🎨VK
4,W60397022,556-206-9662x97397,647.0,2.131239e+14,"Development worker, international aid",jefferyrussell@example.org,http://hicks.net/,Faroe Islands,🏌️,Helen Howell,...,Iwp🏤5!19.865807379370352!G🚵O‍🏿,🍣🏿🟠U👨@-24.53780901409825@️bY‍t,N6♀rs%1.1061886236703802%g2Fu7,dwLNC+-14.192346659192372+D🟠Mj👩,♀Y‍tt^3.9302373419898426^7b🎓🍣T,Q👩👨H🏾?9.83258038734695?p🎨dBV,JC🍣8Q!-0.6727171474082108!Wo‍6t,QtgcM|-7.779141559091242|srwW❇,♀👩🦐pq%15.756719250278918%👨Oet♀,‍🏿🟠👩X+-7.693867142428937+‍b🎨VK
5,536460542,382.430.4085x4188,755.0,3.573601e+15,Higher education lecturer,victoria71@example.com,http://stevens.com/,Holy See (Vatican City State),💢,Lori Ward,...,Iwp🏤5!3.6733795117584935!G🚵O‍🏿,🍣🏿🟠U👨@-25.644481454524406@️bY‍t,N6♀rs%6.3203351298473445%g2Fu7,dwLNC+-5.690021677758599+D🟠Mj👩,♀Y‍tt^-3.410613794664516^7b🎓🍣T,Q👩👨H🏾?-6.971306752531813?p🎨dBV,JC🍣8Q!-1.1264840749074434!Wo‍6t,QtgcM|0.18573819070655162|srwW❇,♀👩🦐pq%-16.02897539206797%👨Oet♀,‍🏿🟠👩X+-1.4251251305758292+‍b🎨VK
6,S24510672,(375)541-9403,457.0,4.943150e+12,"Production assistant, television",larsongregory@example.org,http://ortega.net/,Guinea,🐷,Jeff Chung,...,Iwp🏤5!5.448322743476875!G🚵O‍🏿,🍣🏿🟠U👨@7.566470010377632@️bY‍t,N6♀rs%-6.024057339565218%g2Fu7,dwLNC+-4.939262270599788+D🟠Mj👩,♀Y‍tt^-3.3100027652710633^7b🎓🍣T,Q👩👨H🏾?-2.319398880712992?p🎨dBV,JC🍣8Q!-5.593253585407148!Wo‍6t,QtgcM|-1.6119017844177228|srwW❇,♀👩🦐pq%-20.2001605157438%👨Oet♀,‍🏿🟠👩X+-3.574013595430962+‍b🎨VK
7,Q77978496,001-703-545-2529x004,424.0,3.450809e+14,Educational psychologist,yeseniafoster@example.org,https://www.hodge-stephens.biz/,Turks and Caicos Islands,💁‍♀️,Robert Hoffman,...,Iwp🏤5!18.044409347246148!G🚵O‍🏿,🍣🏿🟠U👨@26.02630457788923@️bY‍t,N6♀rs%5.616338507075293%g2Fu7,dwLNC+-33.586575098989485+D🟠Mj👩,♀Y‍tt^-1.3313043926934092^7b🎓🍣T,Q👩👨H🏾?3.6558145292902124?p🎨dBV,JC🍣8Q!-4.190071367251685!Wo‍6t,QtgcM|-4.641093201776547|srwW❇,♀👩🦐pq%0.03882419317667911%👨Oet♀,‍🏿🟠👩X+-0.24140144594257706+‍b🎨VK
8,582825242,859.694.6199x1168,5143.0,4.625428e+12,Agricultural engineer,ashl

In [4]:
df_test_UID = df_test['UID']

In [5]:
df_train.shape

(1000, 111)

In [6]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 111 entries, UID to state
dtypes: float64(2), object(109)
memory usage: 867.3+ KB
None


In [7]:
print(df_train.dtypes)

UID                    object
ph_no                  object
cvv                   float64
credit_card_number    float64
job                    object
                       ...   
pZijn                  object
WUc3c                  object
sCIyG                  object
qaERi                  object
state                  object
Length: 111, dtype: object


In [8]:
states = df_train['state'].unique()

In [9]:
map ={}
for i in range(len(states)):
    map[i] = states[i]

map

{0: 'Despair',
 1: 'Worthlessness',
 2: 'Stress',
 3: 'Fatigue',
 4: 'Tears',
 5: 'Nausea',
 6: 'Pain',
 7: 'Death',
 8: 'Anxiety',
 9: 'Dread'}

In [10]:
cat_features = ['job', 'country', 'state']
label_enc = LabelEncoder()
for column in cat_features:
    df_train[column] = label_enc.fit_transform(df_train[column])

df_train.head(10)

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state
0,T77962103,7059868985,673.0,2.131868e+14,104,guerrerodavid@example.org,http://garcia.org/,75,📑,Cathy Cherry,...,🍣🏿🟠U👨@9.028568970658524@️bY‍t,N6♀rs%-0.17374199145886737%g2Fu7,dwLNC+-16.438253859793118+D🟠Mj👩,♀Y‍tt^-5.683708646092431^7b🎓🍣T,Q👩👨H🏾?4.495020766798911?p🎨dBV,JC🍣8Q!1.3976340938324077!Wo‍6t,QtgcM|-2.1616012233562616|srwW❇,♀👩🦐pq%7.619063527183258%👨Oet♀,‍🏿🟠👩X+-7.00274740118153+‍b🎨VK,2
1,O92591533,+1-288-810-2425x013,983.0,4.820055e+15,254,michaela47@example.org,http://ortiz.com/,23,🥷🏿,Austin Graham,...,🍣🏿🟠U👨@-40.77392373961654@️bY‍t,N6♀rs%3.836527507653148%g2Fu7,dwLNC+-20.503862282922828+D🟠Mj👩,♀Y‍tt^4.345119531921794^7b🎓🍣T,Q👩👨H🏾?-0.19904265803833332?p🎨dBV,JC🍣8Q!3.414072326617339!Wo‍6t,QtgcM|8.673900995107994|srwW❇,♀👩🦐pq%-0.6359934104694338%👨Oet♀,‍🏿🟠👩X+-11.039355040136286+‍b🎨VK,9
2,O31883571,791.502.4387x7276,855.0,4.031429e+15,418,zhoffman@example.org,http://kelly.com/,187,👩‍💻,Frank Smith,...,🍣🏿🟠U👨@32.83118091536586@️bY‍t,N6♀rs%6.673810456792015%g2Fu7,dwLNC+31.545003675536307+D🟠Mj👩,♀Y‍tt^-1.1189348865153141^7b🎓🍣T,Q👩👨H🏾?-2.109315804181183?p🎨dBV,JC🍣8Q!6.773526691273227!Wo‍6t,QtgcM|1.7184076239046218|srwW❇,♀👩🦐pq%-35.95525145090523%👨Oet♀,‍🏿🟠👩X+8.6752786215897+‍b🎨VK,7
3,M18080565,2522308761,806.0,3.036423e+13,481,hprice@example.net,http://mann.info/,112,NaN,Austin Young,...,🍣🏿🟠U👨@-11.829454766573882@️bY‍t,N6♀rs%-10.87012476653127%g2Fu7,dwLNC+26.131133464440587+D🟠Mj👩,♀Y‍tt^2.1159115210320385^7b🎓🍣T,Q👩👨H🏾?7.373067390546165?p🎨dBV,JC🍣8Q!-0.2388097363537749!Wo‍6t,QtgcM|-5.42787813792933|srwW❇,♀👩🦐pq%24.501780969213996%👨Oet♀,‍🏿🟠👩X+-0.8456272162320895+‍b🎨VK,4
4,Q74073854,683-521-2001x423,424.0,4.610207e+15,428,michael61@example.net,https://www.mitchell.biz/,176,💆🏿‍♂️,Kimberly Wilson,...,🍣🏿🟠U👨@10.957430264267883@️bY‍t,N6♀rs%-1.5309050040841568%g2Fu7,dwLNC+-17.670060957130605+D🟠Mj👩,♀Y‍tt^0.132418797411758^7b🎓🍣T,Q👩👨H🏾?3.6960467337932292?p🎨dBV,JC🍣8Q!4.588960052905303!Wo‍6t,QtgcM|-3.735829327461521|srwW❇,♀👩🦐pq%18.194146730985878%👨Oet♀,‍🏿🟠👩X+10.404818391481196+‍b🎨VK,8
5,P43985373,219.273.4726x275,428.0,2.131351e+14,221,james93@example.org,https://ochoa-gutierrez.net/,49,🧍🏾‍♀️,Brent Hernandez,...,🍣🏿🟠U👨@17.58268623621198@️bY‍t,N6♀rs%-2.811845523465795%g2Fu7,dwLNC+3.773968437289522+D🟠Mj👩,♀Y‍tt^1.8456705455150582^7b🎓🍣T,Q👩👨H🏾?-2.7994908849184856?p🎨dBV,JC🍣8Q!-1.2433056271592056!Wo‍6t,QtgcM|-2.26513514409478|srwW❇,♀👩🦐pq%-26.64323697122899%👨Oet♀,‍🏿🟠👩X+-1.1054359483286063+‍b🎨VK,5
6,I95928831,601.741.1529x4852,610.0,3.516388e+15,340,michael12@example.net,https://hall.com/,220,🦏,Mr. Todd Richards DDS,...,🍣🏿🟠U👨@-18.588666405251253@️bY‍t,N6♀rs%8.217194776982542%g2Fu7,dwLNC+-3.3867603573540714+D🟠Mj👩,♀Y‍tt^-4.764344688857977^7b🎓🍣T,Q👩👨H🏾?-1.7839891815790612?p🎨dBV,JC🍣8Q!3.1100008893905495!Wo‍6t,QtgcM|3.8792464372230127|srwW❇,♀👩🦐pq%-7.266273863482758%👨Oet♀,‍🏿🟠👩X+-0.761454161839668+‍b🎨VK,8
7,V14371828,505.735.3796x37094,944.0,5.662624e+11,143,youngjordan@example.org,http://www.cardenas.biz/,183,💿,Julie Henderson,...,🍣🏿🟠U👨@12.737872216741858@️bY‍t,N6♀rs%-3.0215000872197977%g2Fu7,dwLNC+-5.372761138249989+D🟠Mj👩,♀Y‍tt^7.167058874093296^7b🎓🍣T,Q👩👨H🏾?1.2475639802668326?p🎨dBV,JC🍣8Q!5.2498269170861125!Wo‍6t,QtgcM|5.767867801335005|srwW❇,♀👩🦐pq%-32.04786386746257%👨Oet♀,‍🏿🟠👩X+1.2147268865928584+‍b🎨VK,6
8,Y13708888,660.780.8534,133.0,3.029480e+13,481,mckinneykathy@example.org,https://www.sanchez-gilbert.com/,96,🤾🏼,Mr. Matthew Goodman,...,🍣🏿🟠U👨@2.8060487179477285@️bY‍t,N6♀rs%-6.085554715936928%g2Fu7,dwLNC+-12.0824224039864+D🟠Mj👩,♀Y‍tt^5.2493706147779315^7b🎓🍣T,Q👩👨H🏾?-2.90735592087888?p🎨dBV,JC🍣8Q!-7.309327293951238!Wo‍6t,QtgcM|2.8859631583166134|srwW❇,♀👩🦐pq%33.30743613439829%👨Oet♀,‍🏿🟠👩X+4.495256093163539+‍b🎨VK,1
9,329524520,7457408571,992.0,3.791713e+14,80,esparzadonna@example.com,https://www.smith.com/,92,🌵,Michael Pollard,...,🍣🏿🟠U👨@-16.779136403773567@️bY‍t,N6♀rs%1.683510900145626%g2Fu7,dwLNC+25.78243062291763+D🟠Mj👩,♀Y‍tt^-2.745780391164773^7b🎓

In [11]:
cat_features = ['job', 'country']
label_enc = LabelEncoder()
for column in cat_features:
    df_test[column] = label_enc.fit_transform(df_test[column])

df_test.head(10)

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,OaMqz,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi
0,675919160,001-869-364-3240x1461,632.0,4.787566e+12,534,lawrencereginald@example.org,http://walker.org/,95,🧜🏻‍♀️,Jean Barajas,...,Iwp🏤5!-21.248937402363666!G🚵O‍🏿,🍣🏿🟠U👨@-5.726684042857591@️bY‍t,N6♀rs%1.5777964549882526%g2Fu7,dwLNC+-8.911004387243423+D🟠Mj👩,♀Y‍tt^5.9810077922437666^7b🎓🍣T,Q👩👨H🏾?-1.2978943388453708?p🎨dBV,JC🍣8Q!-1.6739446957794175!Wo‍6t,QtgcM|1.4464588802972562|srwW❇,♀👩🦐pq%-29.42164549286103%👨Oet♀,‍🏿🟠👩X+1.7950039158754367+‍b🎨VK
1,V09461652,+1-573-696-9623x435,548.0,4.804074e+15,291,jennifer41@example.org,http://shah.com/,152,💁🏻‍♀️,Sandra Ellis,...,Iwp🏤5!-33.934398562714854!G🚵O‍🏿,🍣🏿🟠U👨@-19.149703711482513@️bY‍t,N6♀rs%-2.868717979567546%g2Fu7,dwLNC+15.453724177383293+D🟠Mj👩,♀Y‍tt^-6.21132167380491^7b🎓🍣T,Q👩👨H🏾?-0.9339788851074693?p🎨dBV,JC🍣8Q!0.001376362594596836!Wo‍6t,QtgcM|-4.159290969741395|srwW❇,♀👩🦐pq%-1.5404064055083722%👨Oet♀,‍🏿🟠👩X+1.2969428631934425+‍b🎨VK
2,S75396644,001-755-901-1494x000,808.0,4.890170e+15,312,valdezsheryl@example.net,http://barrett.com/,117,👭🏼,Elizabeth Smith,...,Iwp🏤5!12.391218072873253!G🚵O‍🏿,🍣🏿🟠U👨@-14.048287304034654@️bY‍t,N6♀rs%-5.027959013789114%g2Fu7,dwLNC+6.75452704247651+D🟠Mj👩,♀Y‍tt^7.670390381027927^7b🎓🍣T,Q👩👨H🏾?0.3470814661758477?p🎨dBV,JC🍣8Q!1.407363830373458!Wo‍6t,QtgcM|6.091721410416965|srwW❇,♀👩🦐pq%-22.07040959384532%👨Oet♀,‍🏿🟠👩X+-3.7487716558978468+‍b🎨VK
3,598599835,(625)805-7487x931,9468.0,5.428366e+15,55,NaN,http://melendez.com/,195,🖲️,Allison Rodriguez,...,Iwp🏤5!11.456072353615188!G🚵O‍🏿,🍣🏿🟠U👨@-12.459836898257926@️bY‍t,N6♀rs%-3.4487706404694176%g2Fu7,dwLNC+20.40768203656919+D🟠Mj👩,♀Y‍tt^-1.6018121132255914^7b🎓🍣T,Q👩👨H🏾?1.3550954935361417?p🎨dBV,JC🍣8Q!0.9793080156219274!Wo‍6t,QtgcM|-1.4099964218763166|srwW❇,♀👩🦐pq%2.1205798212068476%👨Oet♀,‍🏿🟠👩X+-3.9241064330693742+‍b🎨VK
4,W60397022,556-206-9662x97397,647.0,2.131239e+14,168,jefferyrussell@example.org,http://hicks.net/,69,🏌️,Helen Howell,...,Iwp🏤5!19.865807379370352!G🚵O‍🏿,🍣🏿🟠U👨@-24.53780901409825@️bY‍t,N6♀rs%1.1061886236703802%g2Fu7,dwLNC+-14.192346659192372+D🟠Mj👩,♀Y‍tt^3.9302373419898426^7b🎓🍣T,Q👩👨H🏾?9.83258038734695?p🎨dBV,JC🍣8Q!-0.6727171474082108!Wo‍6t,QtgcM|-7.779141559091242|srwW❇,♀👩🦐pq%15.756719250278918%👨Oet♀,‍🏿🟠👩X+-7.693867142428937+‍b🎨VK
5,536460542,382.430.4085x4188,755.0,3.573601e+15,292,victoria71@example.com,http://stevens.com/,94,💢,Lori Ward,...,Iwp🏤5!3.6733795117584935!G🚵O‍🏿,🍣🏿🟠U👨@-25.644481454524406@️bY‍t,N6♀rs%6.3203351298473445%g2Fu7,dwLNC+-5.690021677758599+D🟠Mj👩,♀Y‍tt^-3.410613794664516^7b🎓🍣T,Q👩👨H🏾?-6.971306752531813?p🎨dBV,JC🍣8Q!-1.1264840749074434!Wo‍6t,QtgcM|0.18573819070655162|srwW❇,♀👩🦐pq%-16.02897539206797%👨Oet♀,‍🏿🟠👩X+-1.4251251305758292+‍b🎨VK
6,S24510672,(375)541-9403,457.0,4.943150e+12,460,larsongregory@example.org,http://ortega.net/,89,🐷,Jeff Chung,...,Iwp🏤5!5.448322743476875!G🚵O‍🏿,🍣🏿🟠U👨@7.566470010377632@️bY‍t,N6♀rs%-6.024057339565218%g2Fu7,dwLNC+-4.939262270599788+D🟠Mj👩,♀Y‍tt^-3.3100027652710633^7b🎓🍣T,Q👩👨H🏾?-2.319398880712992?p🎨dBV,JC🍣8Q!-5.593253585407148!Wo‍6t,QtgcM|-1.6119017844177228|srwW❇,♀👩🦐pq%-20.2001605157438%👨Oet♀,‍🏿🟠👩X+-3.574013595430962+‍b🎨VK
7,Q77978496,001-703-545-2529x004,424.0,3.450809e+14,188,yeseniafoster@example.org,https://www.hodge-stephens.biz/,224,💁‍♀️,Robert Hoffman,...,Iwp🏤5!18.044409347246148!G🚵O‍🏿,🍣🏿🟠U👨@26.02630457788923@️bY‍t,N6♀rs%5.616338507075293%g2Fu7,dwLNC+-33.586575098989485+D🟠Mj👩,♀Y‍tt^-1.3313043926934092^7b🎓🍣T,Q👩👨H🏾?3.6558145292902124?p🎨dBV,JC🍣8Q!-4.190071367251685!Wo‍6t,QtgcM|-4.641093201776547|srwW❇,♀👩🦐pq%0.03882419317667911%👨Oet♀,‍🏿🟠👩X+-0.24140144594257706+‍b🎨VK
8,582825242,859.694.6199x1168,5143.0,4.625428e+12,26,ashleeerickson@example.org,https://garcia.info/,193,🙅🏼,Jessica Herrera DDS,...,Iwp🏤5!3.0016067760132024!G🚵O‍🏿,🍣🏿🟠U👨@16.388961237203617@️bY‍t,N6♀rs%-6.3431606058848775%g2Fu7,dwLNC+-0.2983337455135953+D🟠Mj👩,♀Y‍tt^-2.401648441466815^7b🎓🍣T,Q👩👨H🏾?-2.8860712595210014?p🎨dBV,JC🍣8Q!-0.16847602338762457!Wo‍6t,QtgcM|2.0024042251058916|srwW❇,♀👩🦐pq%-1

In [12]:
df_train = df_train.replace('[^\d]','',regex=True)
df_test = df_test.replace('[^\d]','',regex=True)
df_train.head()

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state
0,77962103,7059868985,673.0,2.131868e+14,104,,,75,,,...,9028568970658524,601737419914588673727,16438253859793118,56837086460924317,4495020766798911,8139763409383240776,21616012233562616,7619063527183258,700274740118153,2
1,92591533,12888102425013,983.0,4.820055e+15,254,47,,23,,,...,4077392373961654,6383652750765314827,20503862282922828,43451195319217947,019904265803833332,834140723266173396,8673900995107994,06359934104694338,11039355040136286,9
2,31883571,79150243877276,855.0,4.031429e+15,418,,,187,,,...,3283118091536586,6667381045679201527,31545003675536307,111893488651531417,2109315804181183,867735266912732276,17184076239046218,3595525145090523,86752786215897,7
3,18080565,2522308761,806.0,3.036423e+13,481,,,112,NaN,,...,11829454766573882,6108701247665312727,26131133464440587,211591152103203857,7373067390546165,8023880973635377496,542787813792933,24501780969213996,08456272162320895,4
4,74073854,6835212001423,424.0,4.610207e+15,428,61,,176,,,...,10957430264267883,61530905004084156827,17670060957130605,01324187974117587,36960467337932292,845889600529053036,3735829327461521,18194146730985878,10404818391481196,8


In [13]:
for column in df_train.columns:
    try:
        df_train[column] = df_train[column].astype(float)
    except Exception as e:
        df_train.drop(column, axis=1, inplace=True)
        pass

In [14]:
for column in df_test.columns:
    try:
        df_test[column] = df_test[column].astype(float)
    except Exception as e:
        df_test.drop(column, axis=1, inplace=True)
        pass

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 107 entries, UID to state
dtypes: float64(107)
memory usage: 836.1 KB


In [16]:
df_train.fillna(df_train.mean(), inplace=True)
df_train.isna().sum()

UID                   0
ph_no                 0
cvv                   0
credit_card_number    0
job                   0
                     ..
pZijn                 0
WUc3c                 0
sCIyG                 0
qaERi                 0
state                 0
Length: 107, dtype: int64

In [17]:
df_test.fillna(df_test.mean(), inplace=True)
df_test.isna().sum()

UID                   0
ph_no                 0
cvv                   0
credit_card_number    0
job                   0
                     ..
vF2is                 0
pZijn                 0
WUc3c                 0
sCIyG                 0
qaERi                 0
Length: 106, dtype: int64

In [18]:
X = df_train.iloc[:,0:-1]
y = df_train.iloc[:,-1]
# y = y[:].astype('int')

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for column in X.columns:
        X[column] = scaler.fit_transform(X[[column]])

X.head()

In [19]:
bestfeatures = SelectKBest(score_func=chi2, k=10) 
fit = bestfeatures.fit(X, y)

In [20]:
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X.columns)

In [21]:
feature_scores = pd.concat([df_columns, df_scores], axis = 1)
feature_scores.columns = ['Feature', 'Score']

In [22]:
final_features = feature_scores.nlargest(20, 'Score')
final_features = np.asanyarray(final_features['Feature'])
final_features

array(['qLu1T', 'aDdCa', 'sRaqu', 'dTj0P', 'HpyWy', 'pZijn', 'FpCOT',
       'gl2KL', 'GZVIZ', 'EjWMB', '7Jilh', 'yeIIP', 'C3boy', 'NKspI',
       'UZRdX', 'Zavi4', 'XIIc6', 'kMpCl', '5k16L', 'cO1Q3'], dtype=object)

In [23]:
for column in X.columns:
    if column not in final_features:
        X.drop(columns=[column], inplace=True)

X

,yeIIP,5k16L,EjWMB,kMpCl,7Jilh,GZVIZ,C3boy,aDdCa,UZRdX,XIIc6,gl2KL,qLu1T,cO1Q3,sRaqu,NKspI,Zavi4,HpyWy,dTj0P,FpCOT,pZijn
0,9.719122e+16,7.105237e+18,6.594721e+18,9.294211e+17,2.229750e+17,2.207591e+16,9.113597e+17,8.099000e+18,1.499953e+17,9.292604e+17,3.788076e+17,4.067495e+20,1.658237e+18,4.611501e+20,8.213884e+17,8.102932e+17,9.931305e+18,2.284563e+18,6.017374e+20,8.139763e+18
1,9.171800e+17,7.330292e+17,6.535446e+18,9.702613e+17,2.448631e+17,2.046500e+17,9.115080e+17,8.004381e+20,1.107085e+18,9.728307e+16,3.064316e+18,4.415872e+19,6.341407e+16,4.617074e+20,8.396507e+17,8.250896e+17,9.956207e+18,2.201370e+20,6.383653e+18,8.341407e+17
2,9.453016e+17,7.426287e+17,6.563049e+18,9.158887e+18,2.610552e+17,2.349851e+17,9.136347e+17,8.021729e+20,1.663223e+17,9.111342e+17,3.031994e+19,4.582582e+20,6.677353e+16,4.613438e+20,8.311235e+18,8.969153e+16,9.949433e+19,2.221076e+19,6.667381e+18,8.677353e+17
3,9.220286e+16,7.657290e+17,6.552352e+17,9.993720e+17,2.263373e+18,2.125760e+17,9.210408e+16,8.068445e+19,1.003710e+19,9.383204e+15,3.140301e+18,4.244626e+20,6.023881e+17,4.611488e+20,8.337128e+17,8.288519e+17,9.939810e+19,2.238961e+19,6.108701e+18,8.023881e+18
4,9.352667e+16,7.455828e+17,6.549185e+18,9.154126e+18,2.361874e+18,2.621366e+16,1.550651e+18,8.001949e+21,1.351771e+18,9.076982e+17,3.475177e+17,4.470098e+19,6.458896e+16,4.619489e+19,8.440767e+16,8.990561e+16,9.922110e+19,2.247651e+18,6.153091e+19,8.458896e+17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9.088171e+17,7.470479e+17,6.522157e+18,9.080794e+18,2.733172e+17,2.101833e+17,9.982038e+15,8.007725e+20,1.146043e+18,9.503616e+16,3.012407e+19,4.524590e+19,6.582067e+16,4.610547e+19,8.051181e+18,8.179956e+17,9.950786e+18,2.259361e+19,6.313794e+18,8.582067e+17
996,9.194142e+17,7.024211e+18,6.568222e+18,9.069748e+17,2.419151e+16,2.261841e+16,9.432462e+16,8.016844e+20,1.389003e+17,9.585705e+16,3.533781e+17,4.289616e+20,6.596549e+16,4.614909e+19,8.205391e+18,8.217190e+17,9.904206e+19,2.205701e+19,6.394653e+19,8.596549e+17
997,9.218687e+16,7.191445e+17,6.511842e+19,9.055177e+17,2.911104e+16,2.713825e+16,9.996416e+16,8.065146e+19,1.139035e+18,9.522166e+16,3.175031e+14,4.129928e+20,6.149272e+17,4.610492e+20,8.492015e+17,8.173571e+17,9.939631e+18,2.254410e+18,6.451992e+18,8.149272e+18
998,9.118529e+15,7.408968e+17,6.529483e+18,9.117877e+15,2.938778e+16,2.081252e+17,9.199471e+16,8.009786e+19,1.118543e+17,9.159496e+17,3.343876e+17,4.314446e+17,6.650328e+17,4.611992e+18,8.547741e+18,8.193597e+17,9.924854e+19,2.200317e+20,6.745938e+18,8.650328e+18


In [24]:
for column in df_test.columns:
    if column not in final_features:
        df_test.drop(columns=[column], inplace=True)

X

,yeIIP,5k16L,EjWMB,kMpCl,7Jilh,GZVIZ,C3boy,aDdCa,UZRdX,XIIc6,gl2KL,qLu1T,cO1Q3,sRaqu,NKspI,Zavi4,HpyWy,dTj0P,FpCOT,pZijn
0,9.719122e+16,7.105237e+18,6.594721e+18,9.294211e+17,2.229750e+17,2.207591e+16,9.113597e+17,8.099000e+18,1.499953e+17,9.292604e+17,3.788076e+17,4.067495e+20,1.658237e+18,4.611501e+20,8.213884e+17,8.102932e+17,9.931305e+18,2.284563e+18,6.017374e+20,8.139763e+18
1,9.171800e+17,7.330292e+17,6.535446e+18,9.702613e+17,2.448631e+17,2.046500e+17,9.115080e+17,8.004381e+20,1.107085e+18,9.728307e+16,3.064316e+18,4.415872e+19,6.341407e+16,4.617074e+20,8.396507e+17,8.250896e+17,9.956207e+18,2.201370e+20,6.383653e+18,8.341407e+17
2,9.453016e+17,7.426287e+17,6.563049e+18,9.158887e+18,2.610552e+17,2.349851e+17,9.136347e+17,8.021729e+20,1.663223e+17,9.111342e+17,3.031994e+19,4.582582e+20,6.677353e+16,4.613438e+20,8.311235e+18,8.969153e+16,9.949433e+19,2.221076e+19,6.667381e+18,8.677353e+17
3,9.220286e+16,7.657290e+17,6.552352e+17,9.993720e+17,2.263373e+18,2.125760e+17,9.210408e+16,8.068445e+19,1.003710e+19,9.383204e+15,3.140301e+18,4.244626e+20,6.023881e+17,4.611488e+20,8.337128e+17,8.288519e+17,9.939810e+19,2.238961e+19,6.108701e+18,8.023881e+18
4,9.352667e+16,7.455828e+17,6.549185e+18,9.154126e+18,2.361874e+18,2.621366e+16,1.550651e+18,8.001949e+21,1.351771e+18,9.076982e+17,3.475177e+17,4.470098e+19,6.458896e+16,4.619489e+19,8.440767e+16,8.990561e+16,9.922110e+19,2.247651e+18,6.153091e+19,8.458896e+17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9.088171e+17,7.470479e+17,6.522157e+18,9.080794e+18,2.733172e+17,2.101833e+17,9.982038e+15,8.007725e+20,1.146043e+18,9.503616e+16,3.012407e+19,4.524590e+19,6.582067e+16,4.610547e+19,8.051181e+18,8.179956e+17,9.950786e+18,2.259361e+19,6.313794e+18,8.582067e+17
996,9.194142e+17,7.024211e+18,6.568222e+18,9.069748e+17,2.419151e+16,2.261841e+16,9.432462e+16,8.016844e+20,1.389003e+17,9.585705e+16,3.533781e+17,4.289616e+20,6.596549e+16,4.614909e+19,8.205391e+18,8.217190e+17,9.904206e+19,2.205701e+19,6.394653e+19,8.596549e+17
997,9.218687e+16,7.191445e+17,6.511842e+19,9.055177e+17,2.911104e+16,2.713825e+16,9.996416e+16,8.065146e+19,1.139035e+18,9.522166e+16,3.175031e+14,4.129928e+20,6.149272e+17,4.610492e+20,8.492015e+17,8.173571e+17,9.939631e+18,2.254410e+18,6.451992e+18,8.149272e+18
998,9.118529e+15,7.408968e+17,6.529483e+18,9.117877e+15,2.938778e+16,2.081252e+17,9.199471e+16,8.009786e+19,1.118543e+17,9.159496e+17,3.343876e+17,4.314446e+17,6.650328e+17,4.611992e+18,8.547741e+18,8.193597e+17,9.924854e+19,2.200317e+20,6.745938e+18,8.650328e+18


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [26]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 

model =  tree.DecisionTreeClassifier(min_impurity_decrease=1)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.15

In [27]:
predicted = model.predict(df_test)

In [28]:
df_test['state'] = predicted.astype('int')
df_test['state'] = df_test['state'].map(map)
submission = pd.concat([df_test_UID, df_test['state']], axis=1)


submission

,UID,state
0,675919160,Dread
1,V09461652,Dread
2,S75396644,Dread
3,598599835,Dread
4,W60397022,Dread
...,...,...
59995,G18996472,Dread
59996,119545284,Dread
59997,Q57447995,Dread
59998,611053577,Dread


In [29]:
submission.to_csv('Submission_22BAI1213.csv', index=None)